In [4]:
from sgp4.api import Satrec, WGS72
from datetime import datetime
import pyproj

# TLE data for a satellite
line1 = '1 25544U 98067A   20356.54791667  .00016717  00000-0  10270-3 0  9003'
line2 = '2 25544  51.6434  21.4215 0007516  45.8848 314.2827 15.49456387255089'

# Create satellite object
satellite = Satrec.twoline2rv(line1, line2)

# Define time
jd, fr = 2458827.5, 0.7842  # Julian date and fraction
e, r, v = satellite.sgp4(jd, fr)

# ECI coordinates (r is in kilometers)
x_eci, y_eci, z_eci = r

# Convert ECI to ECEF
ecef = pyproj.Proj(proj='geocent', ellps='WGS84', datum='WGS84')
latlon = pyproj.Proj(proj='latlong', ellps='WGS84', datum='WGS84')
transform = pyproj.Transformer.from_proj(ecef, latlon)
longitude, latitude, altitude = transform.transform(x_eci * 1000, y_eci * 1000, z_eci * 1000, radians=False)

print(f"Latitude: {latitude}, Longitude: {longitude}, Altitude: {altitude}")

Latitude: 40.70979147001893, Longitude: 123.6254251506847, Altitude: 429791.76004184224


In [7]:
import numpy as np
import pyproj

# Define the satellite's position and velocity in ECI (example values in km and km/s)
satellite_position_eci = [x_eci, y_eci, z_eci]  # example ECI position in kilometers

# Set the reference time for the position (example: July 1, 2020, at 12:00:00 UTC)
from datetime import datetime
reference_time = datetime(2024, 11, 3, 11, 7, 0)

# Convert ECI to ECEF
eci_to_ecef_transformer = pyproj.Transformer.from_crs("epsg:4978", "epsg:4978", always_xy=True)

# Define the image dimensions and pixel size
image_width = 2048  # pixels
image_height = 2048  # pixels
pixel_size = 288  # meters

# Convert satellite position from kilometers to meters
x_eci, y_eci, z_eci = [coord * 1000 for coord in satellite_position_eci]

# Perform the ECI to ECEF transformation (without time argument)
x_ecef, y_ecef, z_ecef = eci_to_ecef_transformer.transform(x_eci, y_eci, z_eci)

# Define a geodetic coordinate system for lat/lon conversion
geodetic = pyproj.CRS("EPSG:4326")

# Convert ECEF to geodetic (latitude, longitude, altitude)
ecef_to_geodetic_transformer = pyproj.Transformer.from_crs("EPSG:4978", geodetic, always_xy=True)
lon, lat, alt = ecef_to_geodetic_transformer.transform(x_ecef, y_ecef, z_ecef)

# Function to map pixel coordinates to geographic coordinates
def pixel_to_latlon(row, col, lat, lon, pixel_size, image_width, image_height):
    # Calculate the offset from the sub-satellite point
    delta_x = (col - image_width / 2) * pixel_size
    delta_y = (row - image_height / 2) * pixel_size

    # Approximate conversion of meters to degrees
    lat_per_meter = 1 / 111320
    lon_per_meter = 1 / (111320 * np.cos(np.deg2rad(lat)))

    # Apply the offset to the sub-satellite point to get the pixel's latitude and longitude
    pixel_lat = lat + delta_y * lat_per_meter
    pixel_lon = lon + delta_x * lon_per_meter

    return pixel_lat, pixel_lon

# Example: Get latitude and longitude for the center pixel
center_pixel_lat, center_pixel_lon = pixel_to_latlon(image_height / 2, image_width / 2, lat, lon, pixel_size, image_width, image_height)

print(f"Center Pixel Latitude: {center_pixel_lat}, Longitude: {center_pixel_lon}")

# Optionally, you can iterate over all pixels to get their lat/lon
pixel_lat_lon_map = np.zeros((image_height, image_width, 2))
for row in range(image_height):
    for col in range(image_width):
        pixel_lat, pixel_lon = pixel_to_latlon(row, col, lat, lon, pixel_size, image_width, image_height)
        pixel_lat_lon_map[row, col, 0] = pixel_lat
        pixel_lat_lon_map[row, col, 1] = pixel_lon

# Now, pixel_lat_lon_map contains the lat/lon for each pixel in the image


Center Pixel Latitude: 40.53181707822618, Longitude: 123.6254251506847


In [8]:
pixel_lat_lon_map

array([[[ 37.88258963, 120.13980687],
        [ 37.88258963, 120.14321079],
        [ 37.88258963, 120.14661471],
        ...,
        [ 37.88258963, 127.10083166],
        [ 37.88258963, 127.10423559],
        [ 37.88258963, 127.10763951]],

       [[ 37.88517676, 120.13980687],
        [ 37.88517676, 120.14321079],
        [ 37.88517676, 120.14661471],
        ...,
        [ 37.88517676, 127.10083166],
        [ 37.88517676, 127.10423559],
        [ 37.88517676, 127.10763951]],

       [[ 37.8877639 , 120.13980687],
        [ 37.8877639 , 120.14321079],
        [ 37.8877639 , 120.14661471],
        ...,
        [ 37.8877639 , 127.10083166],
        [ 37.8877639 , 127.10423559],
        [ 37.8877639 , 127.10763951]],

       ...,

       [[ 43.17328312, 120.13980687],
        [ 43.17328312, 120.14321079],
        [ 43.17328312, 120.14661471],
        ...,
        [ 43.17328312, 127.10083166],
        [ 43.17328312, 127.10423559],
        [ 43.17328312, 127.10763951]],

       [[ 43.17